# Day 2: Exploring numerical groundwater flow modelling with flopy and MODFLOW

We'll walk step-by-step through the process of building and visualizing a basic groundwater flow model using FloPy — a Python package for working with MODFLOW.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy

print(os.getcwd())

The cell below initialises core inputs for building a MODFLOW 6 groundwater model using FloPy. It includes:

- Model Naming and Paths: Sets a name for the model and identifies the path to the MODFLOW 6 executable (mf6.exe) as well as the working directory where input/output files will be stored.

- Aquifer Geometry and Grid: Defines aquifer characteristics including top elevation, thickness, and the horizontal extent of the model domain. The grid is discretized into 10 vertical layers and a 101×101 cell horizontal grid.

- Initial Conditions and Hydraulic Properties: Specifies starting hydraulic head and a list of hydraulic conductivity values assigned to each model layer.

- Stresses: Includes a pumping rate that will be assigned to a well cell in later steps. Since we will be using minutes as our unit of time we divide the pumping rate (which is in m<sup>3</sup>/d) by 1440 to get it in m<sup>3</sup>/min.

In [ ]:
# Name for model
name = "mfTest"
MF6Path = "C:\\WRDAPP\\mf6.4.4_win64\\bin\\mf6.exe"
sim_ws = os.path.join('./', name)
# Model Parameters
TopElevation = 0.00
AquiferThickness = 50.00
LengthofSides = 404.00
HydraulicHead = 20.00
HydraulicConductivity = [10.00, 8.0, 5.0, 5.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
NumberLayers = 10
NumberGridRow = 101
NumberGridCol = 101
NumberCells = 101
PumpingRate = -86.40 / 1440

In this cell, we create the main simulation object using FloPy's MFSimulation class. This object forms the backbone of our model and must be initialized before adding specific model components. The setup includes:

- Simulation Name (sim_name): Matches the model name defined earlier, ensuring consistent file organization.

- Working Directory (sim_ws): Designates where all input/output files will be stored.

- MODFLOW 6 Executable (exe_name): Tells FloPy where to find the MODFLOW 6 engine needed to run the simulation.

- Verbosity Level (verbosity_level): Controls how much feedback is printed during the simulation (1 = summary output).

- MODFLOW Version (version): Confirms that we are using the correct engine ("mf6") for this simulation.

Note: The commented-out MFSimulation.load() line is useful when returning to an existing simulation. Simply uncomment and run to reload previous work.

In [ ]:
# Define the model simulation (Simulation object must be defined before model objects)
#   > Assigning name (sim_name)
#   > Assigning path to MODFLOW 6 simulation working folder (ws)
#   > Setting level of standard output 0=silent, 1=condensed, 2=fullinfo (verbosity)
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=sim_ws, verbosity_level=1, exe_name=MF6Path, version="mf6")

# Example code for loading a model (just remove the # before the two lines of code below)
#sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=sim_ws, 
#                                  exe_name=MF6Path, version="mf6", verbosity_level=1)

This cell sets up the Temporal Discretization (TDIS) Package in MODFLOW 6, specifying how the simulation time is divided into stress periods and time steps.

Here’s what’s happening:

- Stress Period Definitions: PumpingDuration and RecoveryDuration are each set to 3 days (converted to minutes). This results in a total simulation time of 6 days with two distinct phases.

- Time Step Control: Each stress period includes 54 time steps. A step length multiplier (StepLengthMultiplier = 1.22) controls geometric progression—i.e., time steps grow longer over the course of a stress period to improve computational efficiency while maintaining resolution at early times.

- Each tuple in the ModelTimeSteps structure defines:

    - Stress period length (in minutes)

    - Number of time steps in that period

    - Time step multiplier

- Package Execution: ModflowTdis registers these parameters with the simulation object and sets the time unit to minutes.

In [ ]:
# Create temporal discretization package
# Stress Periods:
PumpingDuration = RecoveryDuration = 3.0 * 1440.0 # 3 days pumping, 3 days recovery, 6 days total
# Timestep length multiplier:
StepLengthMultiplier = 1.22
# Number of stress periods:
ModelTimeStepsPhases = 2
# Modflow Stress Periods composition:
    #   > Value 1: Length of the stress period
    #   > Value 2: Number of time steps in the stress period
    #   > Value 3: Multiplier for the length of successive time steps
ModelTimeSteps = [(PumpingDuration, 54, StepLengthMultiplier), (RecoveryDuration, 54, StepLengthMultiplier)]

flopy.mf6.ModflowTdis(sim, pname="tdis", time_units="MINUTES", nper=ModelTimeStepsPhases, perioddata=ModelTimeSteps)

This cell configures three key components required to define the groundwater flow system:

## 1 - Initialize the Iterative Solver (IMS)

Sets up the IMS package, which controls how MODFLOW solves the system of equations.

We're using:

- "SIMPLE" complexity: suitable for models without complicated convergence behavior.

- "BICGSTAB": Also knows as the "Biconjugate gradient stabilised method", a robust linear acceleration scheme for moderate-to-large models.

## 2 - Create the Groundwater Flow (GWF) Model

The ModflowGwf object is instantiated to simulate saturated groundwater flow.

Parameters:

- modelname: Inherits the simulation name.

- save_flows=True: Ensures that internal cell-by-cell flow data are written to output (useful for water budget and post-processing).

- newtonoptions="NEWTON UNDER_RELAXATION": Enables the Newton-Raphson method with damping, which enhances convergence in nonlinear problems.

## 3 - Calculate Grid Geometry and Discretisation

Calculates:

- bot: Bottom elevations for each layer using np.linspace for uniform distribution from top to base.

- delrow and delcol: Cell dimensions calculated based on model side length and number of grid cells.

The DIS package (ModflowGwfdis) then defines the spatial structure of the model:

- Number of layers (nlay), rows (nrow), and columns (ncol)

- Cell sizes in each direction

- Top and bottom elevations

In [ ]:
# Create the flopy package object
flopy.mf6.ModflowIms(sim, pname="ims", complexity="SIMPLE", linear_acceleration="BICGSTAB")
# Create the flopy groundwater flow object
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True, newtonoptions="NEWTON UNDER_RELAXATION")
# Calculate bottom, column and row size
bot = np.linspace(-AquiferThickness / NumberLayers, -AquiferThickness, NumberLayers)
delrow = delcol = LengthofSides / (NumberCells - 1)
# Define discretization of model
flopy.mf6.ModflowGwfdis(gwf, length_units="METERS", nlay=NumberLayers, nrow=NumberGridRow, delr=delrow, ncol=NumberGridCol, delc=delcol, top=TopElevation, botm=bot)

This cell defines two fundamental components of the groundwater model: the initial head distribution and the hydraulic characteristics of the aquifer.

## 1 - Initial Conditions (IC) Package

Creates an array StartingHeads filled with the initial head value (20.00 m) for every cell in the model.

- The shape of the array matches the full 3D grid: [NumberLayers, NumberGridRow, NumberGridCol].

- The ModflowGwfic package uses this array to initialize the starting hydraulic head at every model cell.

## 2 - Node Property Flow (NPF) Package

Governs how water moves through the model by defining:

Hydraulic conductivity for each layer in our model.

- icelltype=1: Enables confined/unconfined flow logic, allowing for variation in saturated thickness.

- save_specific_discharge=True: Stores Darcy velocity results for later visualization or analysis.

- save_flows=True: Ensures internal flow exchanges are recorded between adjacent cells.

In [ ]:
# Create the Initial Conditions (ic) package
#   > CellID = (LAYER, ROW, COLUMN)
StartingHeads = HydraulicHead * np.ones((NumberLayers, NumberGridRow, NumberGridCol))
flopy.mf6.ModflowGwfic(gwf, strt=StartingHeads)

# Create the Node Property Flow (npf) package
# icelltype:
#   0 means saturated thickness is held constant; 
#   >0 means saturated thickness varies with computed head when head is below the cell top; 
#   <0 means saturated thickness varies with computed head unless the THICKSTRT option is in effect. 
#   (When THICKSTRT is in effect, a negative cell type indicates that saturated thickness will be computed as STRT-BOT and held constant.)
flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True, save_flows=True, k=HydraulicConductivity, icelltype=1)

This cell defines the boundary conditions for the model using MODFLOW’s Constant Head (CHD) Package, which maintains a fixed hydraulic head at specified cells throughout the simulation.

## 1 - Create the Boundary Condition List

A list chd_rec is constructed by looping through:

- Each layer in the model

- Each row/column index along the model's edges

Assigned boundary cells:

- Left and right boundaries: All cells in the first (col = 0) and last (col = NumberGridRow - 1) columns

- Top and bottom boundaries: All cells in the first and last rows—excluding the corners already assigned by the columns

Each boundary cell is assigned a constant hydraulic head equal to the predefined HydraulicHead (20.00 m).

## 2 - Apply to Both Stress Periods

The same set of boundary conditions (chd_rec) is applied to both stress periods (0 = pumping phase, 1 = recovery phase) using the dictionary chd_spd.

## 3 - Register the CHD Package

ModflowGwfchd binds this constant head data to the groundwater flow model (gwf).

In [ ]:
# Define the constant head boundary conditions
chd_rec = []
for layer_number in range(NumberLayers): 
    for row_col in range(0, NumberGridRow):
        chd_rec.append(((layer_number, row_col, 0), HydraulicHead))
        chd_rec.append(((layer_number, row_col, NumberGridRow - 1), HydraulicHead))
        if row_col != 0 and row_col != NumberGridRow - 1:
            chd_rec.append(((layer_number, 0, row_col), HydraulicHead))
            chd_rec.append(((layer_number, NumberGridRow - 1, row_col), HydraulicHead))

# Apply the constant heads to both stress periods
chd_spd = {0: chd_rec, 1: chd_rec}

# Create the Constant Head (chd) package
flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_rec)

This cell sets up the Well (WEL) Package in MODFLOW 6 to simulate groundwater extraction during the model’s first stress period.

## 1 - Define Pumping Schedule (WelPeriods)

This dictionary assigns well activity across two stress periods:

Stress Period 0 (Pumping Phase):

- A single well is placed in the bottom layer at the center cell of the model domain.

- The abstraction rate is defined by PumpingRate (negative sign indicates extraction).

Stress Period 1 (Recovery Phase):

- No pumping occurs (empty list), allowing the system to recover hydraulically.

## 2 - Register the WEL Package

The ModflowGwfwel object attaches this schedule to the groundwater flow model.

- stress_period_data=WelPeriods ensures MODFLOW adjusts well fluxes according to the defined timeline.

In [ ]:
# Define the well stress periods
WelPeriods = {
    0: [(NumberLayers - 1, int(NumberGridRow/2), int(NumberGridCol/2), float(PumpingRate))],
    1: [],
}
wel = flopy.mf6.mfgwfwel.ModflowGwfwel(gwf, stress_period_data=WelPeriods)

This final setup cell defines the Output Control (OC) Package, which tells MODFLOW what simulation results to save and where to store them. Output control is essential for tracking head distributions and internal flow behavior during model execution.

## 1 - Define Output Filenames

headfile and budgetfile assign names for binary output files:

- *.hds: Stores computed hydraulic heads.

- *.cbb: Stores detailed cell-by-cell flow information (e.g., intercell flows and stress package exchanges).

## 2 - Set Output Instructions for Each Stress Period

saverecord is a dictionary indicating what to save at each stress period:

- "HEAD", "ALL": Save head values at every time step.

- "BUDGET", "ALL": Save full budget information at every time step.

This ensures both pumping and recovery phases are fully recorded for post-processing and analysis.

## 3 - Register the OC Package

The ModflowGwfoc object binds these output preferences to the simulation.

In [ ]:
# Create the Output Control (oc) package
headfile = "{}.hds".format(name)
head_filerecord = [headfile]
budgetfile = "{}.cbb".format(name)
budget_filerecord = [budgetfile]
saverecord = {0: [("HEAD", "ALL"), ("BUDGET", "ALL")], 
              1: [("HEAD", "ALL"), ("BUDGET", "ALL")]}
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    saverecord=saverecord,
    head_filerecord=head_filerecord,
    budget_filerecord=budget_filerecord
)
print("Done creating simulation.")

This cell finalizes the model setup by saving all configuration files to the working directory using FloPy’s write_simulation() method.

What it does:

- Traverses all model and package objects created within the sim (simulation) container.

- Converts each into the appropriate MODFLOW 6 input file format.

- Writes them to the designated simulation workspace (sim_ws), ready for execution.

After this step, your working directory will contain all model input files (*.nam, *.dis, *.npf, etc.), which you can run via Python or directly from the command line using MODFLOW 6.

In [ ]:
# Writes the model and associated files to disk
sim.write_simulation()

This cell executes the MODFLOW 6 model using the simulation object (sim) and ensures the process completes successfully.

Step-by-step:

- sim.run_simulation() triggers MODFLOW to run using the input files previously written to disk.

- success: A Boolean flag indicating whether the model terminated normally (True) or failed (False).

- buff: A list containing the text output from the MODFLOW run (useful for diagnostics if needed).

If success is False, an exception is raised with a custom message, helping you catch and respond to failed simulations during model creation. Wrapping the execution in a conditional check like this helps prevent silent failures and improves reproducibility.

In [ ]:
success, buff = sim.run_simulation()
if not success:
    raise Exception("MODFLOW 6 did not terminate normally.")

This cell introduces a custom plotting function GridPlot() to visualize the model grid, boundary conditions, and a designated cross-sectional line.

## 1 - Function Overview

Figure Setup:

- Creates a square figure (12×12) with equal axis scaling for spatial accuracy.

- Adds a title showing the model name and highlights the layer being visualized.

Cross-Section Line Definition:

- Defines a horizontal red dashed line at mid-row height (for plotting vertical profiles later).

- Coordinates are calculated dynamically using model dimensions for generality.

Map View Rendering:

- Uses flopy.plot.PlotMapView() to generate a map of Layer 9 (indexing starts at 0).

Plots:

- Constant Head (CHD) boundaries (plot_bc(name='chd'))

- Well (WEL) locations (plot_bc(name='wel'))

- Grid lines (plot_grid()), helping visualize the spatial discretization.

In [ ]:
def GridPlot(gwf):
    # let's take a look at our grid before making a cross section
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(1, 1, 1, aspect="equal")
    ax.set_title(f"{name} Model Grid with cross sectional line\nLayer: {NumberLayers}")

    line = np.array([( 0, (delrow * NumberGridRow)/2 ), ( ((delcol * NumberGridCol)/2), ((delrow * NumberGridRow)/2) ), 
                    ( (delcol * NumberGridCol), ((delrow * NumberGridRow)/2) )])

    mapview = flopy.plot.PlotMapView(model=gwf, layer=9)

    chd = mapview.plot_bc(name='chd', kper=0) # Plots constant head boundary conditions

    wel = mapview.plot_bc(name='wel', kper=0) # Plots well boundary condition
    
    linecollection = mapview.plot_grid() # Plots grid lines
    linecrossection = plt.plot(line.T[0], line.T[1], "r--", lw=2) # Plots cross=sectional line

# Display model area and features
GridPlot(gwf)

This cell defines and executes a custom function LayerParametersPlot() that plots a 2D vertical cross section of the model's horizontal hydraulic conductivity distribution (KxyLayers). This helps you to visually verify layering, property assignments, and boundary alignments in the x-direction.

## Function Breakdown

Extract Data:

- gwf.npf.k.array retrieves the assigned k values from the NPF package as a 3D array (layers × rows × columns).

Set Cross Section Line:

- Constructs a horizontal transect at mid-domain row height, spanning the full model width (west to east).

- Defined in map units using delrow and NumberGridRow.

Plot Using FloPy's CrossSection Tools:

- Initialises PlotCrossSection.

- plot_ibound(): Displays active/inactive model cells (can act as a sanity check).

- plot_array(KxyLayers): Shades cells based on horizontal conductivity (darker = lower K).

- plot_grid(): Overlays cell edges for clarity.

Colorbar & Labels:

- Adds a colorbar labeled in meters per day.

- Title indicates the nature and orientation of the cross-section.

Export:

- Saves the plot as a .jpg image file for documentation or reporting.

In [ ]:
def LayerParametersPlot(gwf):
    KxyLayers = gwf.npf.k.array

    fig, ax = plt.subplots(1, figsize=(40, 8))

    line = np.array( [ ( 0.0, (delrow * NumberGridRow)/2 ), 
                        ( (delrow * NumberGridRow), ((delrow * NumberGridRow)/2) ) ] )
    
    csp = flopy.plot.PlotCrossSection(gwf, ax=ax, line={"line": line})
    boundaries = csp.plot_ibound()
    patch_collection = csp.plot_array(KxyLayers)
    line_collection = csp.plot_grid(colors='white')

    cbar = plt.colorbar(patch_collection, shrink=0.75)
    cbar.ax.set_xlabel("Horizontal Hydraulic Conductivity, ($m/d$)")

    ax.set_title(f"{name} Hydraulic Conductivity West-East Crossection")
    plt.savefig(f"{name}_CSP.jpg")

# Display K-values in cross-section
LayerParametersPlot(gwf)

This cell defines Basic2DFlowPlot(), a function that creates a 2D map of hydraulic heads, flow directions, and boundary conditions.

## What’s Visualised:

Hydraulic Head Distribution:

- head = gwf.output.head().get_data(kstpkper=(0, 0)) extracts heads from the first stress period and time step.

- plot_array(head) shades each cell by its head value.

- contour_array(head) overlays head contour lines.

Flow Vector Field:

- Retrieves specific discharge using the DATA-SPDIS budget entry.

- Calculates vector components (qx, qy) and plots direction arrows to represent groundwater movement.

- Vectors are normalized and colored white for visibility.

Boundary Conditions (Optional):

- If PlotCBC=True, the function also shows constant head boundaries. This is especially helpful for model diagnostics — though in small models, they may clutter the plot.

Styling & Output:

- Enlarged figure size and colorbar for clarity.

- Grid lines and equal axis scaling preserve spatial fidelity.

- Saves a .jpg image to the /output folder.

In [ ]:
def Basic2DFlowPlot(gwf, PlotCBC=False):
    head = gwf.output.head().get_data(kstpkper=(0, 0))
    budget = gwf.output.budget()

    fig, ax = plt.subplots(1, figsize=(20, 18))

    spdis = budget.get_data(text='DATA-SPDIS')[NumberLayers - 1]
    qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)
    pmv = flopy.plot.PlotMapView(gwf, ax=ax)
    patch_collection = pmv.plot_array(head)
    pmv.plot_grid(colors='white')
    pmv.contour_array(head, linewidths=3.)
    cbar = plt.colorbar(patch_collection, shrink=0.75)
    cbar.ax.set_ylabel("Head (m)")
    pmv.plot_vector(qx, qy, normalize=True, color="white") # Plot flow lines
    if PlotCBC == True:
        pmv.plot_bc(name='chd', kper=0) # Plots constant head boundary conditions. Works, but isn't pretty on small models.
    plt.title(f"{name}")
    # Set equal x and y axis.
    plt.axis("equal")
    plt.savefig(f".\output\{name}_2dflow.jpg")

# Display results
if success:
    Basic2DFlowPlot(gwf, PlotCBC=True)

This cell defines CrossectionPlot(), a function that generates a vertical west–east cross-sectional plot of simulated hydraulic heads and flow directions beneath the centerline of the model domain.

## Key Steps in the Function:

Head & Flow Data Extraction:

- Retrieves head data from the first stress period and time step.

- Accesses the specific discharge (flow vector) data from the model’s budget output using 'DATA-SPDIS'.

- Calculates qx, qy, and qz components using FloPy’s built-in post-processing tool.

Cross Section Setup:

- Constructs a horizontal line at the midpoint of the model’s y-axis (center row), spanning west to east.

- Sets up the FloPy cross-section plotting object along this transect.

Plot Components:

- plot_array(head): Shades each grid cell by its simulated hydraulic head.

- plot_grid(): Overlays white gridlines to delineate model cells.

- plot_vector(qx, qy, qz): Draws flow vectors representing both magnitude and direction of groundwater movement through the cross section.

- Vectors are normalized for visual clarity.

- Parameters hstep=9 and kstep=1 adjust the density of arrows in horizontal and vertical directions.

- Enhance with Colorbar & Title:

- A colorbar clarifies the head values in meters.

- Title includes the model name and cross-section orientation for documentation.

Save the Output:

- Exports the plot as a .jpg image.

In [ ]:
def CrossectionPlot(gwf):
    head = gwf.output.head().get_data(kstpkper=(0, 0))
    budget = gwf.output.budget()
    spdis = budget.get_data(text='DATA-SPDIS')[0]
    qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf, head=head)

    fig, ax = plt.subplots(1, figsize=(40, 8))

    line = np.array( [ ( 0.0, (delrow * NumberGridRow)/2 ), 
                        ( (delrow * NumberGridRow), ((delrow * NumberGridRow)/2) ) ] )
    
    csp = flopy.plot.PlotCrossSection(gwf, ax=ax, line={"line": line})
    patch_collection = csp.plot_array(head, head=head)
    line_collection = csp.plot_grid(colors='white')

    cbar = plt.colorbar(patch_collection, shrink=0.75)
    cbar.ax.set_ylabel("Head, ($m$)")

    quiver = csp.plot_vector(qx, qy, qz, head=head, normalize=True, hstep=9, kstep=1) # Plot flow lines

    ax.set_title(f"{name} West-East Crossection")
    plt.savefig(f"{name}_CS.jpg")

# Display results
if success:
    CrossectionPlot(gwf)

# Exercise:

Play around with the code and create your own box model. You can change parameters, boundary conditions, well placement and rates, etc. Use this opportunity to make sure you understand (not memorise) the code and ask any questions you may have.

Suggestion: Save a copy of the file so that you don't overwrite the initial file and can continue using it as a reference.